# Pract – 3: Implementation of Federated Learning Using Flower

### Objective
- Use Flower framework for Federated Learning
- Create 3 clients with synthetic datasets
- Train Linear Regression model locally
- Aggregate models using Flower server
- Evaluate global model


## Step 1: Install Required Libraries

In [1]:
# %pip install flwr scikit-learn numpy

## Step 2: Import Libraries

In [2]:
import flwr as fl
import numpy as np
from sklearn.linear_model import LinearRegression

## Step 3: Create Synthetic Client Datasets

In [3]:
# Client 1 dataset
X1 = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y1 = 2 * X1.flatten()

# Client 2 dataset
X2 = np.array([2, 4, 6, 8, 10]).reshape(-1, 1)
y2 = 2 * X2.flatten()

# Client 3 dataset
X3 = np.array([3, 6, 9, 12, 15]).reshape(-1, 1)
y3 = 2 * X3.flatten()

## Step 4: Define Flower Client

In [4]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, X, y):
        self.model = LinearRegression()
        self.X = X
        self.y = y
        self.model.fit(X, y)

    def get_parameters(self, config):
        return [self.model.coef_, np.array([self.model.intercept_])]

    def set_parameters(self, parameters):
        self.model.coef_ = parameters[0]
        self.model.intercept_ = parameters[1][0]

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.fit(self.X, self.y)
        return self.get_parameters(config), len(self.X), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss = np.mean((self.model.predict(self.X) - self.y) ** 2)
        return loss, len(self.X), {}

## Step 5: Create Clients

In [5]:
clients = [
    FlowerClient(X1, y1),
    FlowerClient(X2, y2),
    FlowerClient(X3, y3)
]

## Step 6: Start Federated Training Simulation

In [6]:
def client_fn(cid):
    return clients[int(cid)]

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=3)
)

INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2026-02-09 12:27:16,801	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'object_store_memory': 790096281.0, 'node:127.0.0.1': 1.0, 'memory': 1580192564.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 3

History (loss, distributed):
	round 1: 0.0
	round 2: 0.0
	round 3: 0.0

(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=14444) 
(ClientAppActor pid=14444)         
(ClientAppActor pid=12104) 
(ClientAppActor pid=12104)         
(ClientAppActor pid=12104) 
(ClientAppActor pid=12104)         
(ClientAppActor pid=12104) 
(ClientAppActor pid=12104)         
(ClientAppActor pid=12104) 
(ClientAppActor pid=12104)         
(ClientAppActor pid=12104) 
(ClientAppActor pid=12104)         


## Step 7: Test Global Model

In [7]:
coef = np.mean([c.model.coef_ for c in clients])
intercept = np.mean([c.model.intercept_ for c in clients])

def global_predict(X):
    return coef * X + intercept

test = np.array([[7], [14]])
print(global_predict(test))

[[14.]
 [28.]]
